## Step 0: Setup Spark

### Configure Spark for Your Notebook
* You may need to Reconnect and/or Restart the Kernel to pick up changes.
* This examples uses the local Spark Master `--master local[1]`
* In production, you would use the PipelineIO Spark Master `--master spark://apachespark-master-2-1-0:7077`

In [164]:
import os

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
conf = '--conf spark.cores.max=1 --conf spark.executor.memory=512m'
packages = '--packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'
jars = '--jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar'
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + conf \
  + ' ' + packages \
  + ' ' + jars \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

--master local[1] --conf spark.cores.max=1 --conf spark.executor.memory=512m --packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1 --jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar --py-files /root/lib/jpmml.py pyspark-shell


### Import Spark Libraries

In [165]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

### Create Spark Session
This may take a minute or two.  Please be patient.

In [166]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Step 1: Load Training Data into Spark Cluster

### Read Data from Public S3 Bucket
* AWS credentials are not needed.
* We're asking Spark to infer the schema
* The data has a header
* Using `bzip2` because it's a splittable compression file format

In [167]:
df = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")

print(df.head())

Row(id=5731498, name='A 2-bdrm house in Plaka of Athens', space='Ideally located a unique house in a very peaceful neighborhood of Plaka, near Acropolis. It is a traditional house in the heart of the historical center of Athens, in Plaka. The kitchen is fully equipped with oven, fridge with freezer. Cutlery, dishes and pans, kettle, espresso coffee maker (espresso capsules are provided), toaster. There is also a vacuum cleaner and a laundry machine. One big closet will make your stay more comfortable. Bed linen, towels and bath amenities are provided. Moreover, the apartment is fully airconditioned. The apartment is very close to a greek traditional tavernas, a pharmacy, banks and public transport.  Airport or any other transport is available upon demand at an additional but very reasonable cost. ', price='120.0', bathrooms='1.0', bedrooms='2.0', room_type='Entire home/apt', square_feet=None, host_is_super_host='0.0', city='Athina', state=None, cancellation_policy='moderate', security_

In [168]:
print(df.count())

198576


### Clean, Filter, and Summarize the Data

In [169]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+
|summary|      square_feet|             price|          bedrooms|         bathrooms|     cleaning_fee|
+-------+-----------------+------------------+------------------+------------------+-----------------+
|  count|           151864|            151864|            151864|            151864|           151864|
|   mean|545.5920823895063|131.00769109202972|1.3336998893747036|1.1988786019069695|37.25118527103198|
| stddev|363.2346181084825| 89.59372969879449|0.8460907193971746|0.4836515839573332|  42.625502170779|
|    min|            104.0|              50.0|               0.0|               0.5|              0.0|
|    max|          32292.0|             750.0|              10.0|               8.0|            700.0|
+-------+-----------------+------------------+------------------+------------------+-----------------+



In [170]:
print(df_final.count())

151864


In [171]:
print(df_final.schema)

StructType(List(StructField(id,IntegerType,true),StructField(city,StringType,true),StructField(state,StringType,true),StructField(space,StringType,true),StructField(price,DoubleType,true),StructField(bathrooms,DoubleType,true),StructField(bedrooms,DoubleType,true),StructField(room_type,StringType,true),StructField(host_is_super_host,StringType,true),StructField(cancellation_policy,StringType,true),StructField(security_deposit,DoubleType,true),StructField(price_per_bedroom,StringType,true),StructField(number_of_reviews,DoubleType,true),StructField(extra_people,DoubleType,true),StructField(instant_bookable,StringType,true),StructField(cleaning_fee,DoubleType,true),StructField(review_scores_rating,DoubleType,true),StructField(square_feet,DoubleType,true)))


In [172]:
# Most popular cities

spark.sql("""
    select 
        state,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by state
    order by count(*) desc
""").show()

+------+-----+------------------+---------+
| state|   ct|         avg_price|max_price|
+------+-----+------------------+---------+
| Other|87467|122.00503046863389|    750.0|
|    NY|22899| 145.9446264028997|    750.0|
|    CA|20750|157.40173493975902|    750.0|
|Berlin| 6034|  80.6433543254889|    650.0|
|    IL| 3552|141.46903153153153|    690.0|
|    TX| 3108|195.25611325611325|    750.0|
|    WA| 2700| 131.4962962962963|    750.0|
|    DC| 2590|136.64015444015445|    720.0|
|    OR| 1954|114.02661207778915|    700.0|
|London|  810|108.84444444444445|    600.0|
+------+-----+------------------+---------+



In [173]:
# Most expensive popular cities

spark.sql("""
    select 
        city,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by city
    order by avg(price) desc
""").filter("ct > 25").show()

+-------------------+---+------------------+---------+
|               city| ct|         avg_price|max_price|
+-------------------+---+------------------+---------+
|         Palm Beach| 26| 348.7692307692308|    701.0|
|        Watsonville| 38| 313.3157894736842|    670.0|
|             Malibu|136| 280.9852941176471|    750.0|
|             Avalon| 38|262.42105263157896|    701.0|
|           Capitola| 35|             246.4|    650.0|
|           Tamarama| 72|             238.5|    750.0|
|    Manhattan Beach|109|232.10091743119267|    700.0|
|Rancho Palos Verdes| 39|230.02564102564102|    750.0|
|       Avalon Beach| 38|229.60526315789474|    620.0|
|            Newport| 52| 223.8653846153846|    750.0|
|      Darling Point| 29|221.51724137931035|    623.0|
|        Middle Park| 34| 212.7941176470588|    671.0|
|            Balmain| 55|212.56363636363636|    712.0|
|        North Bondi|180|206.68333333333334|    750.0|
|             Bronte|144|203.70833333333334|    750.0|
|        Q

## Step 2: Feature Engineering

In [174]:
continuous_features = ["bathrooms", \
                       "bedrooms", \
                       "security_deposit", \
                       "cleaning_fee", \
                       "extra_people", \
                       "number_of_reviews", \
                       "square_feet", \
                       "review_scores_rating"]

categorical_features = ["room_type", \
                        "host_is_super_host", \
                        "cancellation_policy", \
                        "instant_bookable", \
                        "state"]

### Continuous Features

In [175]:
continuous_feature_assembler = VectorAssembler(inputCols=continuous_features, outputCol="unscaled_continuous_features")

continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", \
                                           withStd=True, withMean=False)

### Categorical Features

In [176]:
categorical_feature_indexers = [StringIndexer(inputCol=x, \
                                              outputCol="{}_index".format(x)) \
                                for x in categorical_features]

categorical_feature_one_hot_encoders = [OneHotEncoder(inputCol=x.getOutputCol(), \
                                                      outputCol="oh_encoder_{}".format(x.getOutputCol() )) \
                                        for x in categorical_feature_indexers]

### Assemble Continuous and Categorical Features

In [177]:
feature_cols_lr = [x.getOutputCol() \
                   for x in categorical_feature_one_hot_encoders]
feature_cols_lr.append("scaled_continuous_features")

feature_assembler_lr = VectorAssembler(inputCols=feature_cols_lr, \
                                       outputCol="features_lr")

## Step 3: Train the Pipeline

In [197]:
linear_regression = LinearRegression(featuresCol="features_lr", \
                                     labelCol="price", \
                                     predictionCol="price_prediction", \
                                     maxIter=10, \
                                     regParam=0.3, \
                                     elasticNetParam=0.8)

estimators_lr = \
  [continuous_feature_assembler, continuous_feature_scaler] \
  + categorical_feature_indexers + categorical_feature_one_hot_encoders \
  + [feature_assembler_lr] + [linear_regression]

pipeline = Pipeline(stages=estimators_lr)

pipeline_model = pipeline.fit(training_dataset)

print(pipeline_model)

PipelineModel_4d899955d65933761722


## Step 4:  Export the Pipeline Model

In [19]:
from jpmml import toPMMLBytes

model = toPMMLBytes(spark, training_dataset, pipeline_model)

with open('airbnb.model', 'wb') as fh:
    fh.write(model)

ImportError: No module named 'jpmml'

## Step 5:  Deploy the Pipeline Model

### Option 1 of 2:  Deploy with PipelineIO CLI!

In [20]:
%%bash

pip install -q pio-cli==0.37

Configure CLI for Model Deployment

In [24]:
%%bash

pio init-model --model-server-url=http://prediction-pmml.demo.pipeline.io --model-type=pmml \
    --model-namespace=cli --model-name=airbnb


("Merging dict '{'model_namespace': 'cli', 'model_input_mime_type': "
 "'application/json', 'model_output_mime_type': 'application/json', "
 "'model_server_url': 'http://prediction-pmml.demo.pipeline.io', 'model_name': "
 "'airbnb', 'model_type': 'pmml'}' with existing config '/root/.pio/config'.")
---
model_input_mime_type: application/json
model_name: airbnb
model_namespace: cli
model_output_mime_type: application/json
model_server_url: http://prediction-pmml.demo.pipeline.io
model_type: pmml
pio_api_version: v1
pio_git_home: https://github.com/fluxcapacitor/pipeline/
pio_git_version: v1.2.0



{'model_input_mime_type': 'application/json',
 'model_name': 'airbnb',
 'model_namespace': 'cli',
 'model_output_mime_type': 'application/json',
 'model_server_url': 'http://prediction-pmml.demo.pipeline.io',
 'model_type': 'pmml',
 'pio_api_version': 'v1',
 'pio_git_home': 'https://github.com/fluxcapacitor/pipeline/',
 'pio_git_version': 'v1.2.0'}



In [25]:
%%bash

pio deploy --model-version=v0 airbnb.model

model_version: v0
model_path: /root/volumes/source.ml/jupyterhub.ml/notebooks/spark/airbnb.model
request_timeout: 600

Deploying model '/root/volumes/source.ml/jupyterhub.ml/notebooks/spark/airbnb.model' to 'http://prediction-pmml.demo.pipeline.io/api/v1/model/deploy/pmml/cli/airbnb/v0'.


Success!

Predict with 'pio predict' or POST to 'http://prediction-pmml.demo.pipeline.io/api/v1/model/deploy/pmml/cli/airbnb/v0'



### Deployment Option 2:  REST API

In [26]:
import requests

deploy_url = 'http://prediction-pmml.demo.pipeline.io/api/v1/model/deploy/pmml/rest/airbnb/v0'

files = {'file': open('airbnb.model', 'rb')}

response = requests.post(deploy_url, files=files)

print("Success! %s" % response.text)

Success! 


## Step 6:  Predict With Deployed Pipeline Model

### Setup Prediction Inputs

In [27]:
import json

data = {"bathrooms":2.0, 
        "bedrooms":2.0, 
        "security_deposit":175.00, 
        "cleaning_fee":25.0, 
        "extra_people":1.0, 
        "number_of_reviews": 2.0, 
        "square_feet": 250.0, 
        "review_scores_rating": 2.0, 
        "room_type": "Entire home/apt", 
        "host_is_super_host": "0.0", 
        "cancellation_policy": "flexible", 
        "instant_bookable": "1.0", 
        "state": "CA" }

json_data = json.dumps(data)

with open('airbnb-predict-inputs.json', 'wt') as fh:
    fh.write(json_data)

### Predict with CLI
Note:  Run again if you see a fallback on the first try. (bug)

In [30]:
%%bash

pio predict --model-version=v0 \
            --model-input-filename=airbnb-predict-inputs.json

model_version: v0
model_input_filename: airbnb-predict-inputs.json
request_timeout: 30

Predicting file 'airbnb-predict-inputs.json' with model 'pmml/cli/airbnb/v0' at 'http://prediction-pmml.demo.pipeline.io/api/v1/model/predict/pmml/cli/airbnb/v0'...

'{"results":[[{\'price\': \'140.15038088835155\'}]]}'



### Predict with REST

In [31]:
import json

# Note:  You may need to run this twice.
#        A fallback will trigger the first time. (Bug)
predict_url = 'http://prediction-pmml.demo.pipeline.io/api/v1/model/predict/pmml/rest/airbnb/v0'

headers = {'content-type': 'application/json'}

response = requests.post(predict_url, 
                         data=json_data, 
                         headers=headers)

print(response.text)

{"results":[[{'price': '140.15038088835155'}]]}


## Step 7:  Monitor Model Servers through Dashboards

### Fallbacks and Circuit Breaker [Dashboard](http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D)

In [206]:
%%html

<iframe width=800 height=600 src="http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Model%20Servers%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D"></iframe>

### Grafana Prediction Metrics [Dashboard](http://grafana.demo.pipeline.io)

In [207]:
%%html

<iframe width=800 height=600 src="http://grafana.demo.pipeline.io"></iframe>